In [1]:
# Making imports convenient
import sys
import os
PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1, PATH)

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
import transformers
from sklearn.model_selection import StratifiedKFold

from transformers import AutoTokenizer, DataCollatorWithPadding,AutoModelForSequenceClassification,AdamW,get_scheduler,TrainingArguments,Trainer,EarlyStoppingCallback

from src.utils.myutils import *
import yaml
from tqdm import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

CS_DATA_PATH = PATH + '/data/CS/processed/CWNC/train.csv'
CONFIG_PATH = PATH + '/src/utils/config.yaml'

BATCH_SIZE = 64
transformers.utils.logging.set_verbosity_error()

### CWNC train_test split (SKIP IF DONE)

In [2]:
cwnc = load_dataset("csv", data_files=PATH + '/data/CS/raw/CWNC/wiki2.csv')['train']

Using custom data configuration default-f1a8b67b43197a24
Reusing dataset csv (/home/horyctom/.cache/huggingface/datasets/csv/default-f1a8b67b43197a24/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


In [3]:
cwnc = cwnc.train_test_split(0.15,seed=42)

Loading cached split indices for dataset at /home/horyctom/.cache/huggingface/datasets/csv/default-f1a8b67b43197a24/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-b76da9e56131115a.arrow and /home/horyctom/.cache/huggingface/datasets/csv/default-f1a8b67b43197a24/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-ae880ece4b310786.arrow


In [4]:
cwnc['train'].to_csv(PATH + '/data/CS/processed/CWNC/train.csv',index=False)
cwnc['test'].to_csv(PATH + '/data/CS/processed/CWNC/test.csv',index=False) #THIS IS FOR THE FINAL MODEL SELECTED,TUNED

142620

## Load data

In [5]:
data = load_dataset('csv',data_files = CS_DATA_PATH)['train']
data

Using custom data configuration default-6f7c93c8687304cb


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/horyctom/.cache/huggingface/datasets/csv/default-6f7c93c8687304cb/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


Dataset({
    features: ['sentence', 'label'],
    num_rows: 4962
})

In [6]:
with open(CONFIG_PATH) as f:
    config_data = yaml.load(f, Loader=yaml.FullLoader)

## Training

In [7]:
def compute_metrics_eval(eval_preds):
    metric = load_metric("f1")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [9]:
training_args = TrainingArguments(
    output_dir = './',
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,  
    logging_steps=50,
    disable_tqdm = False,
    warmup_steps=10,
    save_total_limit=2,
    weight_decay=0.1,
    learning_rate=5e-5)

### Cross-Val all models

In [13]:
model_scores = {}

for model_name in tqdm(config_data['models']):
    scores = []
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False,padding=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    token_full = preprocess_data(data,tokenizer,'sentence')

    print("Running 5-fold CV on model: ",model_name,"...")
    for train_index, val_index in skfold.split(token_full['input_ids'],token_full['label']):

        token_train = Dataset.from_dict(token_full[train_index])
        token_valid = Dataset.from_dict(token_full[val_index])


        model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
        model.to(device);
        trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,tokenizer=tokenizer);
        trainer.train();

        #evaluation
        eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
        scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
        
    print("Done.")
    model_scores[model_name] = scores


  0%|          | 0/6 [00:00<?, ?it/s]loading configuration file https://huggingface.co/UWB-AIR/Czert-B-base-cased/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/1109a10013d6f221417ff52a1198bdc115db12e68d957a317c3cc463b17f5d43.28e3a761a3c5e32166573d4cc31b0a71ae1b56a98381799f8ff5332f26600fd7
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/UWB-AIR/Czert-

Running 5-fold CV on model:  UWB-AIR/Czert-B-base-cased ...


loading configuration file https://huggingface.co/UWB-AIR/Czert-B-base-cased/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/1109a10013d6f221417ff52a1198bdc115db12e68d957a317c3cc463b17f5d43.28e3a761a3c5e32166573d4cc31b0a71ae1b56a98381799f8ff5332f26600fd7
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/UWB-AIR/Czert-B-base-cased/resolve/main/pytorch_model.bin from cache at /home/horyctom/.cache/huggingface/transformers/6e

Step,Training Loss
50,0.657100
100,0.496400
150,0.394400




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/UWB-AIR/Czert-B-base-cased/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/1109a10013d6f221417ff52a1198bdc115db12e68d957a317c3cc463b17f5d43.28e3a761a3c5e32166573d4cc31b0a71ae1b56a98381799f8ff5332f26600fd7
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "

...

Step,Training Loss
50,0.578700
100,0.426900
150,0.367500




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/DeepPavlov/bert-base-bg-cs-pl-ru-cased/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/969927d8dc3ef116510e05257d0982836522039e0a62148f115b2cd116f6dafb.e8f15c5aad2f4653e46ceeba0bb32c02a629d106a902c964bce60523d290ac8f
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  

Step,Training Loss
50,0.571000
100,0.435600
150,0.370100




Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 6/6 [40:28<00:00, 404.70s/it]

Done.


In [14]:
model_scores

{'UWB-AIR/Czert-B-base-cased': [0.7411883182275931,
  0.7381671701913394,
  0.7086693548387096,
  0.7116935483870968,
  0.7278225806451613],
 'ufal/robeczech-base': [0.7452165156092648,
  0.7573011077542801,
  0.7610887096774194,
  0.7560483870967742,
  0.7701612903225806],
 'bert-base-multilingual-cased': [0.7341389728096677,
  0.7250755287009063,
  0.7328629032258065,
  0.7227822580645162,
  0.7560483870967742],
 'fav-kky/FERNET-C5': [0.5025176233635448,
  0.7653575025176235,
  0.7237903225806451,
  0.7237903225806451,
  0.7318548387096774],
 'fav-kky/FERNET-News': [0.5025176233635448,
  0.756294058408862,
  0.5030241935483871,
  0.5030241935483871,
  0.7449596774193549],
 'DeepPavlov/bert-base-bg-cs-pl-ru-cased': [0.7311178247734139,
  0.7260825780463243,
  0.7197580645161291,
  0.7298387096774195,
  0.7389112903225806]}

In [15]:
for model in config_data['models']:
    print(model,"F1 score:",np.mean(model_scores[model]))

UWB-AIR/Czert-B-base-cased F1 score: 0.72550819445798
ufal/robeczech-base F1 score: 0.7579632020920639
bert-base-multilingual-cased F1 score: 0.7341816099795342
fav-kky/FERNET-C5 F1 score: 0.6894621219504271
fav-kky/FERNET-News F1 score: 0.6019639492577071
DeepPavlov/bert-base-bg-cs-pl-ru-cased F1 score: 0.7291416934671735
